### 맛집 데이터 파악 후 샘플링 작업

- 약 48만 개의 데이터를 크롤링하기엔 시간이 부족하여 2000개만 샘플링

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

plt.rc('font',family='Malgun Gothic')

In [4]:
df = pd.read_csv('con_data/consume.csv')
df

,CMPNM_NM,ADSTRD_NM,MLSFC_NM,SCLAS_NM,JJINHBT_SALES_PRICE_RATE,JJINHBT_SALES_CO_RATE,OTSD_SALES_PRICE_RATE,OTSD_SALES_CO_RATE,ALL_SALES_PRICE_RATE,ALL_SALES_CO_RATE
0,한국맥도날드(유)제주노형점,노형동,패스트푸드,햄버거,21.36,101.96,0.69,19.92,3.23,75.39
1,버거킹제주이마트점,노형동,패스트푸드,햄버거,6.55,28.89,0.24,6.61,1.02,22.19
2,에이바우트커피아이파크점,노형동,음료,커피,1.32,11.63,0.04,1.49,0.19,7.93
3,투썸플레이스 제주노형오거리점,노형동,음료,커피,2.38,10.95,0.12,2.76,0.40,8.64
4,(유)아웃백스테이크하우스코리아제주점,노형동,양식,패밀리 레스토랑,17.50,10.23,0.59,1.92,2.66,7.50
...,...,...,...,...,...,...,...,...,...,...
484430,봄봄,아라동,음료,커피,2.61,19.95,0.13,3.14,0.44,9.59
484431,돈삼겹노형점,연동,한식,돼지고기,3.52,1.89,0.03,0.06,0.46,0.70
484432,그늘집,대륜동,한식,단품요리 전문,1.67,2.10,0.15,0.49,0.36,1.15
484433,강실장회포차,대천동,한식,단품요리 전문,2.09,1.30,0.41,0.68,0.66,1.05


In [5]:
df.columns

Index(['CMPNM_NM', 'ADSTRD_NM', 'MLSFC_NM', 'SCLAS_NM',
       'JJINHBT_SALES_PRICE_RATE', 'JJINHBT_SALES_CO_RATE',
       'OTSD_SALES_PRICE_RATE', 'OTSD_SALES_CO_RATE', 'ALL_SALES_PRICE_RATE',
       'ALL_SALES_CO_RATE'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484435 entries, 0 to 484434
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   CMPNM_NM                  484435 non-null  object 
 1   ADSTRD_NM                 484435 non-null  object 
 2   MLSFC_NM                  484435 non-null  object 
 3   SCLAS_NM                  484435 non-null  object 
 4   JJINHBT_SALES_PRICE_RATE  484435 non-null  float64
 5   JJINHBT_SALES_CO_RATE     484435 non-null  float64
 6   OTSD_SALES_PRICE_RATE     484435 non-null  float64
 7   OTSD_SALES_CO_RATE        484435 non-null  float64
 8   ALL_SALES_PRICE_RATE      484435 non-null  float64
 9   ALL_SALES_CO_RATE         484435 non-null  float64
dtypes: float64(6), object(4)
memory usage: 37.0+ MB


In [7]:
grouped =df.groupby(['CMPNM_NM', 'ADSTRD_NM','MLSFC_NM',
                     'SCLAS_NM'])[['JJINHBT_SALES_PRICE_RATE', 'JJINHBT_SALES_CO_RATE',
                                   'OTSD_SALES_PRICE_RATE', 'OTSD_SALES_CO_RATE', 
                                   'ALL_SALES_PRICE_RATE','ALL_SALES_CO_RATE']].mean()
grouped #4개의 컬럼을 기준으로 6개의 컬럼의 평균값을 구함(월 별로 비율값이 모두 다르기 때문에 음식점을 중심으로 평균값을 구함)

JJINHBT_SALES_PRICE_RATE  \
CMPNM_NM         ADSTRD_NM MLSFC_NM SCLAS_NM                             
(30)년할매닭발노형점     노형동       한식       단품요리 전문                   1.483333   
(makk)           일도1동      간식       베이커리                      0.184000   
(사복)한라원장애인직업재활시설 애월읍       간식       베이커리                      0.112857   
                           베이커리.디저트 베이커리                      0.100000   
(유)다올            예래동       한식       가정식                       0.222593   
...                                                                ...   
힘찬장어             연동        한식       장어                        0.834545   
힙앤롤              구좌읍       한식       단품요리 전문                   0.084000   
힛업               노형동       양식       양식                        0.482500   
힛잇업(HEAT IT UP)  이도1동      음료       커피                        0.147143   
                           카페       커피                        0.165000   

                                              JJINHBT_SALES_CO_RATE  \
CMPNM_NM         ADSTRD_NM MLSFC_NM SCLAS_NM                          
(30)년할매닭발노형점     노형동       한식       단품요리 전문                1.585000   
(makk)           일도1동      간식       베이커리                   0.420000   
(사복)한라원장애인직업재활시설 애월읍       간식       베이커리                   0.074762   
                           베이커리.디저트 베이커리                   0.135000   
(유)다올            예래동       한식       가정식                    0.356667   
...                                                             ...   
힘찬장어             연동        한식       장어                     0.193182   
힙앤롤              구좌읍       한식       단품요리 전문                0.236667   
힛업               노형동       양식       양식                     0.086786   
힛잇업(HEAT IT UP)  이도1동      음료       커피                     0.507857   
                           카페       커피                     0.550000   

                                              OTSD_SALES_PRICE_RATE  \
CMPNM_NM         ADSTRD_NM MLSFC_NM SCLAS_NM                          
(30)년할매닭발노형점     노형동       한식       단품요리 전문                0.015000   
(makk)           일도1동      간식       베이커리                   0.030000   
(사복)한라원장애인직업재활시설 애월읍       간식       베이커리                   0.000476   
                           베이커리.디저트 베이커리                   0.000000   
(유)다올            예래동       한식       가정식                    0.020000   
...                                                             ...   
힘찬장어             연동        한식       장어                     0.070000   
힙앤롤              구좌읍       한식       단품요리 전문                0.009333   
힛업               노형동       양식       양식                     0.045357   
힛잇업(HEAT IT UP)  이도1동      음료       커피                     0.002857   
                           카페       커피                     0.010000   

                                              OTSD_SALES_CO_RATE  \
CMPNM_NM         ADSTRD_NM MLSFC_NM SCLAS_NM                       
(30)년할매닭발노형점     노형동       한식       단품요리 전문             0.106667   
(makk)           일도1동      간식       베이커리                0.454000   
(사복)한라원장애인직업재활시설 애월읍       간식       베이커리                0.000952   
                           베이커리.디저트 베이커리                0.000000   
(유)다올            예래동       한식       가정식                 0.217037   
...                                                          ...   
힘찬장어             연동        한식       장어                  0.068636   
힙앤롤              구좌읍       한식       단품요리 전문             0.121333   
힛업               노형동       양식       양식                  0.049643   
힛잇업(HEAT IT UP)  이도1동      음료       커피                  0.096429   
                           카페       커피                  0.115000   

                                              ALL_SALES_PRICE_RATE  \
CMPNM_NM         ADSTRD_NM MLSFC_NM SCLAS_NM                         
(30)년할매닭발노형점     노형동       한식       단품요리 전문               0.108333   
(makk)           일도1동      간식       베이커리                  0.038000   
(사복)한라원장애인직업재활시설 애월읍       간식       베이커리                  0.008571   


In [8]:
grouped1 = grouped[(grouped['JJINHBT_SALES_CO_RATE'])>(grouped['OTSD_SALES_CO_RATE'])]
# 위의 조건식으로 외지인보다 현지인이 더 많이 방문하는 가게를 뽑아냄
grouped1['JJINHBT_SALES_CO_RATE'].describe()
#뽑은 데이터의 개수,평균,분산,4분위수,최대값을 알 수 있음

count    27211.000000
mean         1.007825
std          2.344974
min          0.003333
25%          0.156154
50%          0.405833
75%          0.985000
max        100.000000
Name: JJINHBT_SALES_CO_RATE, dtype: float64

In [9]:
grouped2 = grouped1[(grouped1['JJINHBT_SALES_CO_RATE']>=0.156154)&(grouped1['JJINHBT_SALES_CO_RATE']<0.985000)]
# IQR(25%~75% 사이에 있는 50%의 가운데 값)을 사용하기 위해 조건식으로 뽑아냄 

In [10]:
grouped3 = grouped2.reset_index() 
grouped3 # 아까 4개의 기준으로 그룹화하였기 때문에 4개의 컬럼이 index가 되었었는데 분석을 용이하게 하기 위해 index를 해제

,CMPNM_NM,ADSTRD_NM,MLSFC_NM,SCLAS_NM,JJINHBT_SALES_PRICE_RATE,JJINHBT_SALES_CO_RATE,OTSD_SALES_PRICE_RATE,OTSD_SALES_CO_RATE,ALL_SALES_PRICE_RATE,ALL_SALES_CO_RATE
0,(유)다올,예래동,한식,가정식,0.222593,0.356667,0.020000,0.217037,0.034815,0.320741
1,(유)케이디에셋 담앤루,중문동,한식,가정식,0.120870,0.331304,0.024783,0.321739,0.031739,0.379565
2,(주)구푸디해운대밀면(본점),대천동,한식,가정식,0.190000,0.350000,0.006667,0.040000,0.013333,0.120000
3,(주)구푸디해운대밀면(본점),대천동,한식,면류,0.126667,0.223333,0.037333,0.210667,0.042667,0.258000
4,(주)그랜드부민,이도2동,한식,가정식,44.770000,0.370000,0.810000,0.030000,5.320000,0.180000
...,...,...,...,...,...,...,...,...,...,...
13598,힘찬장어,연동,한식,단품요리 전문,0.728750,0.238750,0.030000,0.041250,0.076250,0.117500
13599,힘찬장어,연동,한식,장어,0.834545,0.193182,0.070000,0.068636,0.127273,0.125000
13600,힙앤롤,구좌읍,한식,단품요리 전문,0.084000,0.236667,0.009333,0.121333,0.014667,0.184667
13601,힛잇업(HEAT IT UP),이도1동,음료,커피,0.147143,0.507857,0.002857,0.096429,0.013571,0.255000


In [11]:
grouped4 = grouped3.sample(n=2000,replace=False,ignore_index=True)
# 개수 2000개, 중복값 허용 없음, 인덱스 무시하고 새로 생성하는 샘플링
grouped4

,CMPNM_NM,ADSTRD_NM,MLSFC_NM,SCLAS_NM,JJINHBT_SALES_PRICE_RATE,JJINHBT_SALES_CO_RATE,OTSD_SALES_PRICE_RATE,OTSD_SALES_CO_RATE,ALL_SALES_PRICE_RATE,ALL_SALES_CO_RATE
0,청춘두루치기,노형동,한식,가정식,0.392500,0.362500,0.002500,0.012500,0.052500,0.135000
1,원풍해장국,조천읍,한식,가정식,0.665000,0.841250,0.020000,0.151250,0.060000,0.425000
2,돼지,일도2동,아시아음식,중식,0.110000,0.200000,0.000000,0.000000,0.010000,0.110000
3,홍리짬뽕,동홍동,한식,중식,0.301818,0.540000,0.004545,0.040000,0.035455,0.240909
4,돌담집,오라동,한식,단품요리 전문,0.300000,0.295000,0.025000,0.035000,0.065000,0.135000
...,...,...,...,...,...,...,...,...,...,...
1995,삼성웰스토리 (주)호텔신라제주면세점,연동,한식,구내식당/푸드코트,0.340000,0.365000,0.000000,0.002500,0.025000,0.152500
1996,제주돈애월점,애월읍,한식,돼지고기,0.697273,0.259091,0.150000,0.206364,0.217273,0.268182
1997,가을남자의커피향기,애월읍,한식,단품요리 전문,0.090000,0.252500,0.007500,0.147500,0.013750,0.216250
1998,1970새마을포차 제주시청점,이도2동,주점및주류판매,포장마차,0.958571,0.652143,0.028571,0.120000,0.085714,0.297857


In [12]:
grouped4.to_csv('con_data/맛집_sample.csv',index=False)